In [1]:
// Imports
import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection 
import org.apache.spark.sql.functions._

//import scala.sqlContext.implicits._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.SaveMode

import org.apache.spark.input.PortableDataStream
import com.amazonaws.auth.BasicAWSCredentials

//import org.apache.spark.sql.cassandra._

//import com.datastax.spark.connector._
import org.apache.spark.sql.types.IntegerType


import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1611248756501_0003,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.SaveMode
import org.apache.spark.input.PortableDataStream
import com.amazonaws.auth.BasicAWSCredentials
import org.apache.spark.sql.types.IntegerType
import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}


In [2]:
def fileDownloader(urlOfFileToDownload: String, fileName: String) = {
    val url = new URL(urlOfFileToDownload)
    val connection = url.openConnection().asInstanceOf[HttpURLConnection]
    connection.setConnectTimeout(5000)
    connection.setReadTimeout(5000)
    connection.connect()

    if (connection.getResponseCode >= 400)
        println("error")
    else
        url #> new File(fileName) !!
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fileDownloader: (urlOfFileToDownload: String, fileName: String)Any


In [5]:
val test=fileDownloader("http://data.gdeltproject.org/gdeltv2/masterfilelist.txt", "/tmp/masterfilelist.txt") // save the list file to the Spark Master


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

test: Any = ""


In [8]:
println(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
// English Data
val list_csv = spark.read.format("csv").option("delimiter", " ").
    csv("s3://testfuret/masterfilelist.txt").
    withColumnRenamed("_c0","size").
    withColumnRenamed("_c1","hash").
    withColumnRenamed("_c2","url")

//val list_2020_tot = list_csv.where(col("url").like("%/2020%")) 
val list_2020_tot = list_csv.where(col("url").like("%gkg%")) 
list_2020_tot.show(10,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list_csv: org.apache.spark.sql.DataFrame = [size: string, hash: string ... 1 more field]
list_2020_tot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [size: string, hash: string ... 1 more field]
+--------+--------------------------------+---------------------------------------------------------------+
|size    |hash                            |url                                                            |
+--------+--------------------------------+---------------------------------------------------------------+
|10768507|ea8dde0beb0ba98810a92db068c0ce99|http://data.gdeltproject.org/gdeltv2/20150218230000.gkg.csv.zip|
|10269336|2f1a504a3c4558694ade0442e9a5ae6f|http://data.gdeltproject.org/gdeltv2/20150218231500.gkg.csv.zip|
|11279827|66b03e2efd7d51dabf916b1666910053|http://data.gdeltproject.org/gdeltv2/20150218233000.gkg.csv.zip|
|11212939|cd20f295649b214dd16666ca451b9994|http://data.gdeltproject.org/gdeltv2/20150218234500.gkg.csv.zip|
|9728953 |8f4b26e134bd6605cce2d32e92e

In [6]:
def hash(s: String) = {
    val m = java.security.MessageDigest.getInstance("MD5")
    val b = s.getBytes("UTF-8")
    m.update(b, 0, b.length)
    new java.math.BigInteger(1, m.digest()).toString(16)
  }

val hash_fuction=udf(hash _)


def StokeFileS3_Modifier(URL: String) = {
    val fileName = URL.split("/").last
    val dir = "/mnt/tmp/"
    val localFileName = dir + fileName
    try {
            fileDownloader(URL,  localFileName)
            //@transient val s3Client: AmazonS3 = AmazonS3ClientBuilder.defaultClient()
            val localFile = new File(localFileName)
            //s3Client.putObject("testfuret/Master_file", fileName, localFile)   
            //localFile.delete()
        
    } catch {
        case e: java.io.FileNotFoundException => 
        case e: com.amazonaws.SdkClientException => 

    }    
    
}





VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hash: (s: String)String
hash_fuction: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
StokeFileS3_Modifier: (URL: String)Unit


In [9]:
println(SizeEstimator.estimate(localFileName))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:55: error: not found: value SizeEstimator
       println(SizeEstimator.estimate(localFileName))
               ^
<console>:55: error: not found: value localFileName
       println(SizeEstimator.estimate(localFileName))
                                      ^



In [10]:
                                       
list_2020_tot.take(1).foreach(r=> {
                 r.getAs[Int](0) //taille
                 r.getAs[String](1) // hash
                 r.getAs[String](2) // URL
                 fileDownloader(r.getAs[String](2),  "/tmp/test")
    
                  val list_csv = spark.read.format("csv").option("delimiter", " ").
                  csv("/tmp/test").
                  withColumnRenamed("_c0","size").
                  withColumnRenamed("_c1","hash").
                  withColumnRenamed("_c2","url")

                 
                 
               } )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.sql.AnalysisException: Path does not exist: hdfs://ip-172-31-43-223.ec2.internal:8020/tmp/test;
  at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:576)
  at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:559)
  at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
  at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
  at scala.collection.immutable.List.foreach(List.scala:392)
  at scala.collection.TraversableLike$class.flatMap(TraversableLike.scala:241)
  at scala.collection.immutable.List.flatMap(List.scala:355)
  at org.apache.spark.sql.execution.datasources.DataSource.org$apache$spark$sql$execution$datasources$DataSource$$chec

In [14]:
val localfile= new File("/tmp/test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

localfile: java.io.File = /tmp/test


In [ ]:
fileDownloader("http://data.gdeltproject.org/gdeltv2/20150218230000.gkg.csv.zip",  "/tmp/test/localfile")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…